<a href="https://colab.research.google.com/github/PaulinaPacyna/DL_speech_recognition/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You can use this code in your notebook to load the data.

In [1]:
import librosa
import os
import json
import pickle
import zipfile
import numpy as np
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
import pandas as pd
from tqdm.notebook import tqdm 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation, BatchNormalization
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
if not os.path.isdir('train'):
  !pip install kaggle
  !pip install py7zr
  import py7zr
  !mkdir /root/.kaggle
  !touch /root/.kaggle/kaggle.json

  api_token = {"username":"paulinapacyna","key":"31e4c572f1ddefc96883c2195f33726b"}
  # please don't hold it against me xD

  with open('/root/.kaggle/kaggle.json', 'w') as file:
      json.dump(api_token, file)

  !chmod 600 /root/.kaggle/kaggle.json
  !kaggle config path -p /root
  !kaggle competitions download -c tensorflow-speech-recognition-challenge
  with py7zr.SevenZipFile('train.7z', mode='r') as z:
      z.extractall()
  # with py7zr.SevenZipFile('test.7z', mode='r') as z:
  #     z.extractall()


     |████████████████████████████████| 71kB 3.8MB/s 
     |████████████████████████████████| 358kB 10.1MB/s 
     |████████████████████████████████| 122kB 14.0MB/s 
     |████████████████████████████████| 2.2MB 14.4MB/s 
     |████████████████████████████████| 1.9MB 48.0MB/s 
usage: kaggle config [-h] {view,set,unset} ...
kaggle config: error: argument command: invalid choice: 'path' (choose from 'view', 'set', 'unset')
 99% 1.04G/1.04G [00:11<00:00, 124MB/s]
100% 1.04G/1.04G [00:11<00:00, 96.5MB/s]
  0% 0.00/501k [00:00<?, ?B/s]
100% 501k/501k [00:00<00:00, 163MB/s]
 99% 2.44G/2.46G [00:31<00:00, 54.2MB/s]
100% 2.46G/2.46G [00:31<00:00, 83.6MB/s]
  0% 0.00/50.0 [00:00<?, ?B/s]
100% 50.0/50.0 [00:00<00:00, 39.9kB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
paths = [os.path.join(dp, f) for dp, dn, fn in os.walk('train/audio') for f in fn if f[-4:]=='.wav']
paths = pd.DataFrame(paths, columns=['path']) # .sample(n=1000).reset_index(drop=True)
paths['label'] = paths['path'].apply(lambda x: x.split('/')[-2])
paths

,path,label
0,train/audio/five/1a073312_nohash_0.wav,five
1,train/audio/five/c93d5e22_nohash_0.wav,five
2,train/audio/five/57cb3575_nohash_0.wav,five
3,train/audio/five/b5552931_nohash_0.wav,five
4,train/audio/five/187af8be_nohash_0.wav,five
...,...,...
64722,train/audio/on/d90b4138_nohash_0.wav,on
64723,train/audio/on/471a0925_nohash_0.wav,on
64724,train/audio/on/9d050657_nohash_0.wav,on
64725,train/audio/on/d0f7bef5_nohash_0.wav,on


In [28]:
def audio2mel(path):
    y, sr = librosa.core.load(path=path)
    if len(y) > sr: # we set all to have lenght equal to 1 second 
        y = y[:sr] 
    else: # pad blank
        padding = sr - len(y)
        offset = padding // 2 
        y = np.pad(y, (offset, sr - len(y) - offset), 'constant')
    mel = librosa.feature.melspectrogram(y=y, sr=sr)
    return librosa.power_to_db(mel, ref= np.max).astype(np.float) 

def convert_wav_to_image(df):
    X = []
    for _,row in tqdm(df.iterrows(), total = df.shape[0]):
        x = audio2mel(row['path'])
        X.append(x.transpose())
    X = np.array(X) 
    return X
os.chdir('/content/drive/MyDrive/DL3/')
if os.path.exists('X_train.npy') and os.path.exists('y_train.npy') and os.path.exists('class_names.npy'):
  X = np.load('X_train.npy', allow_pickle=True)
  y = np.load('y_train.npy', allow_pickle=True)
  classess = np.load('class_names.npy', allow_pickle=True)
else:
  X = convert_wav_to_image(paths)
  enc = OneHotEncoder() 
  y = enc.fit_transform(paths[['label']]).todense()
  classes = enc.get_feature_names()
  np.save('X_train.npy', X)
  np.save('y_train.npy', y)
  np.save('class_names.npy', classes)
X = (X - np.mean(X))/np.std(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [29]:
model = Sequential()
model.add(BatchNormalization())
model.add(LSTM(
    256,
    input_shape=X.shape[1:],
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(y.shape[1]))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='Adam')

In [30]:
model.fit(X_train, y_train, batch_size=100, epochs=10, validation_split=0.3)

Epoch 1/10
304/304 [==============================] - 18s 38ms/step - loss: 1.9955 - val_loss: 0.6805
Epoch 2/10
304/304 [==============================] - 10s 33ms/step - loss: 0.6330 - val_loss: 0.4829
Epoch 3/10
304/304 [==============================] - 10s 33ms/step - loss: 0.4252 - val_loss: 0.3929
Epoch 4/10
304/304 [==============================] - 10s 33ms/step - loss: 0.3214 - val_loss: 0.3497
Epoch 5/10
304/304 [==============================] - 10s 33ms/step - loss: 0.2654 - val_loss: 0.3303
Epoch 6/10
304/304 [==============================] - 10s 33ms/step - loss: 0.2356 - val_loss: 0.2986
Epoch 7/10
304/304 [==============================] - 10s 33ms/step - loss: 0.1870 - val_loss: 0.3006
Epoch 8/10
304/304 [==============================] - 10s 33ms/step - loss: 0.1688 - val_loss: 0.3212
Epoch 9/10
304/304 [==============================] - 10s 33ms/step - loss: 0.1594 - val_loss: 0.2763
Epoch 10/10
304/304 [==============================] - 10s 33ms/step - loss: 0.138

In [31]:
p = model.predict(X_train)

In [35]:
np.mean(np.argmax(p,1) == np.argmax(y_train,1))

0.9541586920930661

In [ ]:
y